# CS 329 E Data Visualization Homework 5

## National Parks and Biodiversity 

### Submitted by : Amber Etana Vasquez 11am

In our last homework, we'll combine some of the concepts from the 
recent lab (geo data) with merging data and create an interactive 
dashboard. 


In [1]:
# importing packages

# data wrangling
import pandas as pd
import numpy as np

# plotting
import altair as alt

# working with geographic data
import json
!pip install geopandas
import geopandas as gpd
from vega_datasets import data

##✅ Loading in a shape file
This zip file contains a _shape_ file.  This is one of the types of geometry data supported by geopandas ([doc](https://geopandas.org/en/stable/docs/user_guide/io.html?highlight=shape%20files#reading-spatial-data))

In [2]:
# importing shapefile - We can use a zip file that contains a shape file by reading it from a URL
gdf = gpd.read_file('http://cs.utexas.edu/~chaney/nps_boundary.zip')

In [3]:
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry
210,NATR,Lands - http://landsnet.nps.gov/tractsnet/docu...,Natchez Trace Parkway,2016-04-12,MS,SE,123648,Parkway,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Natchez Trace,16.024485,0.020812,"2, 4",SE,"MULTIPOLYGON (((-91.24453 31.63620, -91.24506 ..."
170,BOST,Lands - http://landsnet.nps.gov/tractsnet/docu...,Boston National Historical Park,2015-02-09,MA,NE,600140,National Historical Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Boston,0.055380,0.000019,1,NE,"MULTIPOLYGON (((-71.04573 42.33216, -71.04580 ..."
333,PERL,Lands - http://landsnet.nps.gov/tractsnet/docu...,Pearl Harbor National Memorial,2019-08-29,HI,PW,need,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Pearl Harbor,0.030530,0.000008,12,PW,"MULTIPOLYGON (((-157.95220 21.36360, -157.9521..."
248,JAZZ,Lands - http://landsnet.nps.gov/tractsnet/docu...,New Orleans Jazz National Historical Park,2017-11-08,LA,SE,2035602,National Historical Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,New Orleans Jazz,0.007027,0.000002,4,SE,"POLYGON ((-90.06794 29.96404, -90.06720 29.963..."
359,FIIS,PRELIMINARY - Data has not completed the entir...,Fire Island National Seashore,2013-12-18,NY,NE,970121,National Seashore,Lands,Preliminary data. Contact the Land Resources P...,Fire Island,1.190478,0.008497,1,NE,"MULTIPOLYGON (((-73.01725 40.75431, -73.01727 ..."


## ✅Q1 - Overlay the National Parks and Monuments on a county map of the USA
Using the geopandas data frame `gdf`, filter out just the national parks and monuments and overlay
them on a county map of the USA.  Use the Vega data set to map the counties, using [this page](https://altair-viz.github.io/gallery/choropleth.html) as a reference.  Create a tool tip that shows the park name (`UNIT_NAME`) and region (`REGION`) of the park, title your chart, and encode the color with the Region of the park using the `dark2` colorscheme. See [this page](https://altair-viz.github.io/user_guide/customization.html#customizing-colors) for a reference on changing the color scheme. 

This would be an appropriate visualization if our user wanted to understand the landsize of the parks and how the regions were labeled. 

In [4]:
from vega_datasets import data

gdf.geometry= gdf.geometry.simplify(.001)

height = 300
width = 500

In [5]:
counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

basemap= alt.Chart(counties).mark_geoshape(fill='white',stroke='lightgrey').encode(
).project(
    type='albersUsa'
).properties(
    width=width,
    height=height,
    title="National Parks and Monuments in the USA"
)

natparks = gdf[gdf['UNIT_TYPE'].isin(['National Park'])]
monuments = gdf[gdf['UNIT_TYPE'].isin(['National Monument'])]

parks = alt.Chart(natparks).mark_geoshape().encode(
    tooltip=['UNIT_NAME','REGION'],
    color=alt.Color("REGION", scale=alt.Scale(scheme='dark2'))
).project(
    type='albersUsa'
).properties(
    height=height,
    width=width
)

mons = alt.Chart(monuments).mark_geoshape().encode(
    tooltip=['UNIT_NAME','REGION'],
    color=alt.Color("REGION", scale=alt.Scale(scheme='dark2'))
).project(
    type='albersUsa'
).properties(
    height=height,
    width=width
)

basemap + mons + parks


alt.LayerChart(...)

##✅ Loading in a csv a species data

In [6]:
df_species = pd.read_csv('http://cs.utexas.edu/~chaney/numspecies.csv')
df_species = df_species.fillna( value = 0 ) # Fill the missing species counts with zeros for visualization
df_species.sample(5)

,UNIT_CODE,All Types,Amphibian,Bird,Fish,Mammal,Reptile,Vascular Plant,Fungi,Insect,Invertebrate,Spider/Scorpion,Nonvascular Plant,Algae,Crab/Lobster/Shrimp,Slug/Snail
8,CARE,1566,6.0,248,15.0,73,21.0,1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,DEVA,4439,71.0,531,9.0,212,90.0,2258,123.0,903.0,18.0,32.0,22.0,60.0,38.0,72.0
2,BADL,1389,10.0,239,24.0,64,13.0,625,173.0,239.0,1.0,1.0,0.0,0.0,0.0,0.0
27,HALE,2580,3.0,44,6.0,15,10.0,1000,66.0,1061.0,37.0,46.0,227.0,0.0,18.0,47.0
14,DENA,1320,1.0,179,14.0,43,0.0,841,29.0,51.0,3.0,0.0,159.0,0.0,0.0,0.0


## ✅Q2 - Change data to Long format 
Remember [long form vs wide form formats](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data)? It turns out our species data is wide form.  Convert it to long form and save the new dataframe to `df_species_long`. Name your new columns `Species Type` and `Species Count` so they look pretty when we plot them later.

In [7]:
df_species_long = df_species.melt('UNIT_CODE', var_name='Species Type', value_name='Species Count')



In [8]:
# sniff check
df_species_long.sample(5)

,UNIT_CODE,Species Type,Species Count
482,KOVA,Insect,0.0
400,CARE,Fungi,0.0
739,CONG,Crab/Lobster/Shrimp,6.0
793,CAVE,Slug/Snail,0.0
7,CANY,All Types,1223.0


## ✅Q3 - Get lat/long and area from park shape
We have the shape file for the national parks, which contains the polygon for the park boundary, but we decide for the visualization we want to encode the location with a circle whose size is constant.  We also are interested in comparing the area of the park with the species diversity to discover trends. To accomplish these tasks we need to calculate the centroid of the park to get a single lat/long and calculate the area of the polygon in acres (a convenient unit for the USA). 

To do math in geopandas you have to translate between "EPSG:4269" and "EPSG:5070" (USA Albers projection)

Add the `Acres` column to the `gdf` geopandas dataframe, and replace the `geometry` column with the centroid. Use the [intro to geopandas](https://geopandas.org/en/stable/getting_started/introduction.html) documentation to help with the syntax.

Check that everything looks OK by plotting the lat/long of the parks on top of the county map as before. Include a tooltip that shows the region, name of park, and Acres. Use the `dark2` color scheme.

I'm helping you out with the coordinate reference systems here; note that there are many many map reference coordinates and we can see the reference for a given geopandas dataframe with the `crs` property. 

In [9]:
# look at the coordinate reference of the data - this needs to be translated to do math, but then put back
gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [10]:
gdf = gdf.to_crs( 'EPSG:5070') 

In [11]:
gdf.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
# Find the area in acres
# 0.000247105
#gdf['Acres'] =  # your code here to calculate acrage 
gdf['Acres'] =  (gdf['geometry'].area*0.000247).astype(int)

In [13]:
# sniff check
# The value from Rocky Mountain National Park webpage is 265,807 acres and our estimate is off probably due to the 
# projection we used was not locally optimal, or maybe the shape file wasn't exact. If we were a GIS class
# this would probably bother us, but for now we'll call this close enough (your number should be about 0.5% off)
gdf[gdf['UNIT_CODE'] == 'ROMO']['Acres']

415    266919
Name: Acres, dtype: int64

In [14]:
# gdf = gdf.rename(columns={'geometry': 'Acres'}).set_geometry('Acres')
# gdf['geometry'] =  # your code here to convert the polygons data to point data 

gdf['geometry'] =  gdf.centroid


In [15]:
gdf = gdf.to_crs( 'EPSG:4269') # This takes us back to lat/long space
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry,Acres
366,DEWA,PRELIMINARY - Data has not completed the entir...,Delaware Water Gap National Recreation Area,2018-05-25,PA,NE,1193711,National Recreation Area,Lands,Preliminary data. Contact the Land Resources P...,Delaware Water Gap,4.892573,0.029920,1,NE,POINT (-74.95171 41.12646),68844
252,STRI,Lands - http://landsnet.nps.gov/tractsnet/docu...,Stones River National Battlefield,2018-03-29,TN,SE,1888964,National Battlefield,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Stones River,0.193380,0.000291,2,SE,POINT (-86.43079 35.87580),722
405,HSTR,Lands - http://landsnet.nps.gov/tractsnet/docu...,Harry S Truman National Historic Site,2020-11-30,MO,MW,2463113,National Historic Site,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Harry S Truman,0.014986,0.000005,4,MW,POINT (-94.51560 38.92816),9
194,FRLA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Frederick Law Olmsted National Historic Site,2016-03-01,MA,NE,1971618,National Historic Site,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Frederick Law Olmsted,0.012175,0.000003,1,NE,POINT (-71.13209 42.32419),6
120,KICA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Kings Canyon National Park,2011-10-17,CA,PW,255948,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Kings Canyon,3.154838,0.187763,10,PW,POINT (-118.59796 36.89145),458841


In [16]:
# Check that everything looks OK by plotting the lat/long of the parks on top of the county map as before. 
# Include a tooltip that shows the region, name of park, and Acres. Use the dark2 color scheme.

from vega_datasets import data

gdf.geometry= gdf.geometry.simplify(.001)

height = 300
width = 500

In [17]:
counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

basemap= alt.Chart(counties).mark_geoshape(fill='white',stroke='lightgrey').encode(
).project(
    type='albersUsa'
).properties(
    width=width,
    height=height,
    title="National Parks and Monuments in the USA"
)

natparks = gdf[gdf['UNIT_TYPE'].isin(['National Park'])]
monuments = gdf[gdf['UNIT_TYPE'].isin(['National Monument'])]

parks = alt.Chart(natparks).mark_geoshape().encode(
    tooltip=['UNIT_NAME','REGION',"Acres"],
    color=alt.Color("REGION", scale=alt.Scale(scheme='dark2'))
).project(
    type='albersUsa'
).properties(
    height=height,
    width=width
)

mons = alt.Chart(monuments).mark_geoshape().encode(
    tooltip=['UNIT_NAME','REGION','Acres'],
    color=alt.Color("REGION", scale=alt.Scale(scheme='dark2'))
).project(
    type='albersUsa'
).properties(
    height=height,
    width=width
)

basemap + parks + mons

alt.LayerChart(...)

## ✅Q4 - Merge Geo Data and and Species Data
We've been using the `UNIT_NAME` for the park name in the plots, and we want to use that same name, and the acerage
when plotting the species data.  Add these two columns to our `df_species_long` dataframe using a inner join, renaming the `UNIT_NAME` to `Park Name`, and `REGION` to `Region` for pretty 
visualization. The key for the merge is `UNIT_CODE`. Save this new dataframe as `df_species_final`

In [18]:
gdf[['UNIT_CODE', 'UNIT_NAME', 'REGION']]

df_species_final = df_species_long.merge(gdf[['UNIT_CODE','Acres','UNIT_NAME','REGION']], how='inner', on='UNIT_CODE').rename(columns={"UNIT_NAME": "Park Name", "REGION": "Region"})

df_species_final.sample(5)
#df_species_final.shape



,UNIT_CODE,Species Type,Species Count,Acres,Park Name,Region
806,SHEN,Nonvascular Plant,350.0,197763,Shenandoah National Park,NE
630,LAVO,All Types,1797.0,107446,Lassen Volcanic National Park,PW
303,GAAR,Invertebrate,0.0,7521722,Gates of the Arctic National Park,AK
767,ROMO,Bird,277.0,266919,Rocky Mountain National Park,IM
854,WICA,Slug/Snail,25.0,33909,Wind Cave National Park,MW


In [19]:
tmp=gdf[['UNIT_CODE','Acres','UNIT_NAME','REGION']].rename(columns={"UNIT_NAME": "Park Name", "REGION": "Region"})
df_species_final=pd.merge(df_species_long,tmp,how='inner',on=['UNIT_CODE'])

## ✅Q5 - Compare the Acres to Species Count

On a log/log scale compare the total species count (only the rows where "Species Type" is "All Types") with the park size. Encode the region using the same color scale as above. Add a tool tip to see the park name, Species Count, and Acres.

Add a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [20]:
import altair as alt
from vega_datasets import data

#df_species_final[df_species_final['Species Types']=="All Types"]

click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

alt.Chart(df_species_final[df_species_final['Species Type']=="All Types"]).mark_circle(size=60).encode(
    x=alt.X('Acres:Q',scale=alt.Scale(type='log')),
    y=alt.Y('Species Count:Q',scale=alt.Scale(type='log')),
    color=alt.condition(click, alt.Color('Region:N',
                         scale=alt.Scale(scheme='dark2')),
                         alt.value('lightgray')),
    tooltip=['Park Name', 'Species Count', 'Acres']
).properties(
    title="Park Size vs Species Count (log/log scale)"
).add_selection(
    click
)

alt.Chart(...)

## ✅Q6 - Create a rug plot of Species Diversity
Look at how the different Species categores (all Species types that are NOT 'All Types' are distributed
across all the parks.  

Use the same color encoding for the Region as the prior plots. 

Add a `selection_multi` that will highlight the selected data for a park by turning unselected data light gray. 
Also, make the tick a little bigger when it is selected to help find the park in the other categories.

In [21]:
import altair as alt
from vega_datasets import data

click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

alt.Chart(df_species_final).mark_tick(thickness=3).encode(
    tooltip=['Park Name','Species Count','Acres'],
    x='Species Count',
    y=alt.Y('Species Type',title=""),
    color=alt.condition(click, alt.Color('Region:N',
                         scale=alt.Scale(scheme='dark2')),
                         alt.value('lightgray')),
).properties(
    title="Species Count by Type"
).add_selection(
    click)

alt.Chart(...)

##✅ Q7 - Create a sorted bar chart of Species Count with a Region Drop down selector

Create a bar chart with horizontal bars that shows the total species count per park with the color channel encoding the Region.

Add a drop down selector so that you can filter the bar chart by just one region. See [this documentation](https://altair-viz.github.io/gallery/multiple_interactions.html) for an example of adding a drop down menu and using it to filter your selections.

In addition to the drop down, include a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [22]:
# with selection multi, dropdown and tooltip

import altair as alt
from vega_datasets import data

click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

input_dropdown = alt.binding_select(options=['All','NE','IM','MW','SE','PW','AK', ], name='Filter by Region')
selection = alt.selection_single(fields=['Region'], bind=input_dropdown)

bars = alt.Chart(df_species_final).mark_bar().encode(
    tooltip=['Park Name','Species Count','Species Type'],
    x=alt.X('Species Count:Q'),
    y=alt.Y("Park Name:O", title="", sort='-x'),
    color=alt.condition(click, alt.Color('Region:N',
                        scale=alt.Scale(scheme='dark2')),
                        alt.value('lightgray')),
).properties(
    title="Parks Sorted by Species Count").add_selection(
    selection).transform_filter(selection
).add_selection(
    click)


bars


alt.Chart(...)

##✅  Q8 - Bring all the charts together for a Dashboard

Bring all 4 charts and the region filter together.  Any click on one chart should highlight that data in the
other charts.  The colors for the regions should not change depending on the region selection and be consistent across 
all graphs. When plotting your geopandas data frame, filter out only the rows that have species data before making the map. I used the [documentation on customizing titles](https://altair-viz.github.io/user_guide/configuration.html#config-title) to change the font size and name my dashboard (all of the titles in the component graphs I changed to be `subtitles`)

Change your color scheme to `tableau10`.

Hint: There will be only one `selection_multi` that links all the selection highlighting in the charts together.

In [23]:
import altair as alt
from vega_datasets import data

# geoshape plot

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

In [24]:
import altair as alt
from vega_datasets import data

title = alt.Chart(
    {"values": [{"text": "National Park Diversity Dashboard"}]}
).mark_text(size=20).encode(
    text="text:N"
)

click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

# geoshape plot

counties = alt.topo_feature(data.us_10m.url,'counties')
source = data.unemployment.url

basemap= alt.Chart(counties).mark_geoshape(fill='white',stroke='lightgrey').encode(
).project(
    type='albersUsa'
).properties(
    width=900,
    height=700,
    title="National Parks in the USA"
)

natparks = gdf[gdf['UNIT_TYPE'].isin(['National Park'])]

parks = alt.Chart(natparks).mark_geoshape().encode(
    tooltip=['UNIT_NAME','REGION', 'Acres'],
    color=alt.Color("REGION", scale=alt.Scale(scheme='tableau10'),legend=alt.Legend(title="Region"))
).project(
    type='albersUsa'
).properties(
    height=900,
    width=700
).add_selection(
    click)

# horizontal bar plot
#click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

input_dropdown = alt.binding_select(options=['All','NE','IM','MW','SE','PW','AK', ], name='Filter by Region')
selection = alt.selection_single(fields=['REGION'], bind=input_dropdown)

bars = alt.Chart(df_species_final).mark_bar().encode(
    tooltip=['Park Name','Species Count','Species Type'],
    x=alt.X('Species Count:Q'),
    y=alt.Y("Park Name:O", title="", sort='-x'),
    color=alt.condition(click, alt.Color('Region:N',
                        scale=alt.Scale(scheme='tableau10')),
                        alt.value('lightgray'),legend=alt.Legend(title="")),
).properties(
    title="Parks Sorted by Species Count").add_selection(
    selection).transform_filter(selection
).add_selection(
    click)

# rug plot 

#click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

rug = alt.Chart(df_species_final).mark_tick(thickness=3).encode(
    tooltip=['Park Name','Species Count','Acres'],
    x='Species Count',
    y=alt.Y('Species Type',title=""),
    color=alt.condition(click, alt.Color('Region:N',
                         scale=alt.Scale(scheme='tableau10')),
                         alt.value('lightgray'),legend=alt.Legend(title="")),
).properties(
    title="Species Count by Type"
).add_selection(
    click)

# scatter plot
#click = alt.selection_multi(encodings=['color'], fields=['Park Name'])

scatter = alt.Chart(df_species_final[df_species_final['Species Type']=="All Types"]).mark_circle(size=60).encode(
    x=alt.X('Acres:Q',scale=alt.Scale(type='log')),
    y=alt.Y('Species Count:Q',scale=alt.Scale(type='log')),
    color=alt.condition(click, alt.Color('Region:N',
                         scale=alt.Scale(scheme='tableau10')),
                         alt.value('lightgray'),legend=alt.Legend(title="")),
    tooltip=['Park Name', 'Species Count', 'Acres']
).properties(
    title="Park Size vs Species Count (log/log scale)"
).add_selection(
    click
)

alt.vconcat(
    title,
    bars | (basemap + parks & (rug | scatter)) 

).configure_view(
    stroke=None
).configure_concat(
    spacing=5
)


# Graph1 | (Graph 2 & (Graph 3 | Graph 4)) 


alt.VConcatChart(...)